In [7]:
from transformers import CanineTokenizer, CanineForTokenClassification
import torch
import pandas as pd

import textdistance
from tqdm.auto import tqdm, trange
from collections import Counter

In [61]:
from diff_utils import get_aligned_diff

In [2]:
saved_model_name = '../models/canine-c-bak-semisupervised-v1-1' 

In [3]:
tokenizer = CanineTokenizer.from_pretrained(saved_model_name)
model = CanineForTokenClassification.from_pretrained(saved_model_name)

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [4]:
model.cuda();

In [49]:
LABELS_THIS = [c[5:] for c in model.config.id2label.values() if c.startswith('THIS_')]
LABELS_NEXT = [c[5:] for c in model.config.id2label.values() if c.startswith('NEXT_')]

def fix_text(text, verbose=False):
    bx = tokenizer(text, return_tensors='pt', padding=True)
    with torch.inference_mode():
        out = model(**bx.to(model.device))
        n1, n2 =  len(LABELS_THIS), len(LABELS_NEXT)
        ids1 = out.logits[0, :, :n1].view(-1, n1).argmax(-1).tolist()
        ids2 = out.logits[0, :, n1:].view(-1, n2).argmax(-1).tolist()
    result = []
    for c, id1, id2 in zip(' ' + text, ids1, ids2):
        l1, l2 = LABELS_THIS[id1], LABELS_NEXT[id2]
        if l1 == 'KEEP':
            result.append(c)
        elif l1 != 'DELETE':
            result.append(l1)
        if l2 != 'PASS':
            result.append(l2)
    return ''.join(result)

# Analysis

In [8]:
df_orig = pd.read_csv('../data/spellchecker_dataset_split.tsv', sep='\t')

In [9]:
df_orig_dev = df_orig[(df_orig.split=='dev') & (df_orig.edit_max_cldiff <= 3) & (df_orig.edit_max_lendiff <= 1)]
print(df_orig_dev.shape)

(4611, 9)


In [14]:
pd.options.display.max_colwidth = 200

In [58]:
df_orig_dev[(df_orig_dev.distance > 2) & (df_orig.trash2.str.len() < 50)][['trash2', 'clean2']].sample(10)

C:\Users\david\AppData\Local\Temp/ipykernel_21872/194236841.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_orig_dev[(df_orig_dev.distance > 2) & (df_orig.trash2.str.len() < 50)][['trash2', 'clean2']].sample(10)


,trash2,clean2
2162,"Avаnti,*** - тип мығырҙаны Натйаш.","Avаnti, - тип мығырҙаны Натйаш."
15576,Ая* ныс һәм асыу быулыҡтырҙы уны;,Аяныс һәм асыу быулыҡтырҙы уны.
224,"' - Ниңә, ҡәйнеш, улай тиһең?","Ниңә, ҡәйнеш, улай тиһең?"
9492,У йыл дан д ың йөҙө һөрөмләнде.,Уйылдандың йөҙө һөрөмләнде.
18520,"Иптәш Bjраn, Хәкимовҡа ни булды?","Иптәш врач, Хәкимовҡа ни булды?"
4200,Был бит * ' дөрөҫ түгел.,Был бит дөрөҫ түгел.
19006,' - Кем ҡыҙы булаһың һуң?,Кем ҡыҙы булаһың һуң?
7825,Тағы дәресен әҙерләмәгәнме? -ти ул.Тағы?,Тағы дәресен әҙерләмәгәнме? - ти ул. - Тағы?
6775,** Луидор - француз аҡсаһы.,Луидор - француз аҡсаһы.
16991,"Уларҙың ғәскәренә яҙы-, ламы?",Уларҙың ғәскәренә яҙыламы?


In [56]:
dev2 = df_orig_dev.copy()

In [58]:
model.eval();
dev2['fixed'] = [fix_text(text) for text in tqdm(dev2.trash2)]
dev2['change_amount'] = dev2.apply(lambda row: textdistance.levenshtein.distance(row.trash2, row.fixed), axis=1)
dev2['new_diff'] = dev2.apply(lambda row: textdistance.levenshtein.distance(row.clean2, row.fixed), axis=1)

  0%|          | 0/4611 [00:00<?, ?it/s]

In [60]:
print('reduction', 1 - dev2.new_diff.sum() / dev2.distance.sum())

reduction 0.44337527757216877


Compare along specific error types

In [68]:
old_counter = Counter(
    e
    for edits in [
        get_aligned_diff(row.trash2, row.clean2, keep_equiv=False) 
        for row in tqdm(dev2.itertuples(), total=dev2.shape[0])
    ]
    for e in edits 
)

  0%|          | 0/4611 [00:00<?, ?it/s]

In [69]:
new_counter = Counter(
    e
    for edits in [
        get_aligned_diff(row.fixed, row.clean2, keep_equiv=False) 
        for row in tqdm(dev2.itertuples(), total=dev2.shape[0])
    ]
    for e in edits 
)

  0%|          | 0/4611 [00:00<?, ?it/s]

In [93]:
d = pd.DataFrame({
    'old': pd.Series({str(k): v for k, v in old_counter.items()}),
    'new': pd.Series({str(k): v for k, v in new_counter.items()})
}).fillna(0).astype(int)
d['reduction'] = d['new'] - d['old']

Число каких требуемых правок сократилось больше всего?

In [94]:
d.sort_values('reduction', ascending=True).head(20)

,old,new,reduction
"('', ' ')",373,98,-275
"('г', 'ғ')",215,34,-181
"('', '-')",414,235,-179
"('К', 'Ҡ')",177,13,-164
"('һ', 'Һ')",190,85,-105
"(' ', '')",236,142,-94
"('', '- ')",237,149,-88
"('*', '')",60,1,-59
"('Ь', 'Һ')",55,0,-55
"('ә', 'э')",72,22,-50


Число каких требуемых правок возросло больше всего?

In [95]:
d.sort_values('reduction', ascending=False).head(20)

,old,new,reduction
"(' ', '-')",2,52,50
"('ғ', 'г')",30,65,35
"('- ', '')",2,27,25
"('а', 'ә')",2,19,17
"('ы', 'е')",0,15,15
"('', ',')",10,22,12
"('Ҡ', 'К')",3,14,11
"('Ҡ', 'Һ')",0,10,10
"('т', 'ғ')",2,10,8
"('Һ', 'К')",7,15,8


Суммарно, раньше людям нужно было сделать в среднем 0.94 правки на текст, а теперь - 0.53 правки на текст. 

In [97]:
d.sum() / dev2.shape[0]

old          0.945565
new          0.538278
reduction   -0.407287
dtype: float64

# Export

In [100]:
UPLOAD_NAME = "slone/canine-c-bashkir-gec-v1"

In [99]:
model.push_to_hub(UPLOAD_NAME)

CommitInfo(commit_url='https://huggingface.co/slone/canine-c-bashkir-gec-v1/commit/70ea073b3e95b74f80173fc3bb8c377d67da9c86', commit_message='Upload CanineForTokenClassification', commit_description='', oid='70ea073b3e95b74f80173fc3bb8c377d67da9c86', pr_url=None, pr_revision=None, pr_num=None)

In [101]:
tokenizer.push_to_hub(UPLOAD_NAME)

CommitInfo(commit_url='https://huggingface.co/slone/canine-c-bashkir-gec-v1/commit/d9aa21f9d19cc4f390d3fe8cc09e72fd6356dd4e', commit_message='Upload tokenizer', commit_description='', oid='d9aa21f9d19cc4f390d3fe8cc09e72fd6356dd4e', pr_url=None, pr_revision=None, pr_num=None)

# Minimal example

In [1]:
import torch
from transformers import CanineTokenizer, CanineForTokenClassification

tokenizer = CanineTokenizer.from_pretrained('slone/canine-c-bashkir-gec-v1')
model = CanineForTokenClassification.from_pretrained('slone/canine-c-bashkir-gec-v1')
if torch.cuda.is_available():
    model.cuda()

Downloading:   0%|          | 0.00/816 [00:00<?, ?B/s]

c:\users\david\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


Downloading:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/530M [00:00<?, ?B/s]

In [54]:
LABELS_THIS = [c[5:] for c in model.config.id2label.values() if c.startswith('THIS_')]
LABELS_NEXT = [c[5:] for c in model.config.id2label.values() if c.startswith('NEXT_')]

def fix_text(text, boost=0):
    bx = tokenizer(text, return_tensors='pt', padding=True)
    with torch.inference_mode():
        out = model(**bx.to(model.device))
        n1, n2 =  len(LABELS_THIS), len(LABELS_NEXT)
        logits1 = out.logits[0, :, :n1].view(-1, n1)
        logits2 = out.logits[0, :, n1:].view(-1, n2)
        if boost:
            logits1[1:, 0] -= boost
            logits2[:, 0] -= boost
        ids1, ids2 = logits1.argmax(-1).tolist(), logits2.argmax(-1).tolist()
    result = []
    for c, id1, id2 in zip(' ' + text, ids1, ids2):
        l1, l2 = LABELS_THIS[id1], LABELS_NEXT[id2]
        if l1 == 'KEEP':
            result.append(c)
        elif l1 != 'DELETE':
            result.append(l1)
        if l2 != 'PASS':
            result.append(l2)
    return ''.join(result)

Уйылдандың йөҙө һөрөмләнде.
